In [ ]:
import warnings
warnings.filterwarnings("ignore")
import arviz as az
import pandas as pd
import plotly.express as px
import numpy as np
from tbdynamics.camau.calibration.utils import get_bcm, calculate_future_acf_outputs
from tbdynamics.calibration.plotting import plot_output_ranges, plot_trial_output_ranges
from tbdynamics.tools.inputs import load_targets
from tbdynamics.settings import CM_PATH, OUT_PATH, DOCS_PATH
from tbdynamics.constants import QUANTILES
from tbdynamics.camau.constants import indicator_legends, indicator_names
import estival.sampling.tools as esamp
from typing import Dict, Optional, List
from tbdynamics.tools.detect import make_future_acf_scenarios
import pickle

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
# loaded_inference_data = az.from_netcdf(OUT_PATH / 'inference_data1.nc')
# idata = az.from_netcdf(OUT_PATH / 'extracted_idata.nc')
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    "screening_scaleup_shape": 0.3,
    "screening_inflection_time": 1993,
    "clearance_rate": 0.000000000000,
    "cleared_proportion": 0.9,
    # "prop_mixing_same_stratum": 0.75,
    # "incidence_props_pulmonary": 0.76,
    # "incidence_props_smear_positive_among_pulmonary": 0.4,
    # "acf_sensitivity": 0.85
}
targets = load_targets(CM_PATH / "targets.yml")

In [ ]:
idata_raw = az.from_netcdf(OUT_PATH / 'camau/r24082/calib_full_out.nc')

In [ ]:
idata_raw

In [ ]:
burnt_idata = idata_raw.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [ ]:
# outputs = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs.pkl', 'wb') as f:
#      pickle.dump(outputs, f)

In [ ]:
# with open(OUT_PATH /'camau/quant_outputs.pkl', 'rb') as f:
#     outputs = pickle.load(f)
covid_effects = {"detection_reduction": True, "contact_reduction": False}

#     # Base scenario (calculate outputs for all indicators)
bcm = get_bcm(params, covid_effects)
base_results = esamp.model_results_for_samples(idata_extract, bcm).results
base_quantiles = esamp.quantiles_for_results(base_results, QUANTILES)

In [ ]:
# base_quantiles.to_pickle(OUT_PATH / 'camau/output0808.pkl')
# base_quantiles = pd.read_pickle(OUT_PATH / 'camau/output0808.pkl')

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)
plot_output_ranges(base_quantiles,targets,["total_population","adults_popXact3_trial", "adults_popXact3_control"],indicator_names,indicator_legends,1,2010,2025, option = 'camau') #.write_image(DOCS_PATH /'camau/pops.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['notification','percentage_latent_adults', 'school_aged_latentXact3_trial', 'school_aged_latentXact3_control'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/targets.png', scale=3)


In [ ]:
# plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau').write_image(DOCS_PATH /'camau/burden.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['detection_rate'],indicator_names,indicator_legends,1,1980,2025) #.write_image(DOCS_PATH /'camau/detection_rate.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/burden5.png', scale=3)

In [ ]:
plot_trial_output_ranges(base_quantiles,targets,['acf_detectionXact3_trial','acf_detectionXact3_control'],indicator_names,2) #.write_image(DOCS_PATH /'camau/trial_compare.png', scale=3)